In [5]:
import pandas as pd
from data.loader import load_dataset_from_dataframe

In [2]:
filename = "/home/ran/datasets/test-pcam/test.parquet"
df = load_dataset_from_dataframe(filename)

In [3]:
df["image_tensor"][0].shape

torch.Size([3, 96, 96])

In [8]:
df["label"][0]

np.int64(1)

In [6]:
results_file = "/home/ran/datasets/test-pcam/results.parquet"
results_df = pd.read_parquet(results_file, engine="pyarrow")

In [9]:
results_df[["reject", "model", "label"]]

,reject,model,label
0,False,0.483669,1
1,False,0.496775,1
2,False,0.490231,1
3,False,0.471991,1
4,False,0.491534,1
5,False,0.490353,1
6,False,0.466364,0
7,False,0.494559,1
8,False,0.457188,1
9,False,0.508298,1
